In [ ]:
#%pip install duckdb polars pyiceberg numpy sqlalchemy pyarrow

In [2]:
import duckdb
import polars as pl
from pyiceberg.catalog.sql import SqlCatalog
import uuid
from datetime import date
import numpy as np
import pyarrow

In [3]:
warehouse_path = "./icehouse"
catalog = SqlCatalog(
    "default",
    **{
        "uri": f"sqlite:///{warehouse_path}/icyhot.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

In [ ]:
namespace = "dummy_ns"
catalog.create_namespace(f"{namespace}")

In [5]:
rows = 5000

#duckdb
sql = f"""
    select t.row_id, uuid() as txn_key, current_date as rpt_dt
        ,round(random() * 100,2) as some_val
    from generate_series(1,{rows}) t(row_id)
"""

duck_df = duckdb.execute(sql).arrow()

In [8]:
table_ducks = catalog.create_table(f"{namespace}.duckdb_data",schema = duck_df.schema)

In [9]:
table_ducks.append(duck_df)

/Users/matthewmartin/dream_machine/python/pyiceberg_test/.venv/lib/python3.12/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.name() function instead.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))
/Users/matthewmartin/dream_machine/python/pyiceberg_test/.venv/lib/python3.12/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Support for parsing catalog level identifier in Catalog identifiers is deprecated. Please refer to the table using only its namespace and its table name.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))
/Users/matthewmartin/dream_machine/python/pyiceberg_test/.venv/lib/python3.12/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier

In [10]:
cn = duckdb.connect()
cn.execute("install iceberg; load iceberg;")

In [14]:
sql = f"""
select *
from iceberg_scan('./icehouse/{namespace}.db/duckdb_data')
"""
cn.sql(sql).show()

IOException: IO Error: Iceberg metadata file not found for table version '"1"' using 'none' compression and format(s): 'v%s%s.metadata.json,%s%s.metadata.json'